In [39]:
#!/usr/bin/env python
# coding: utf-8

#Daniel Delgado
#On my honor, I have neither given nor received unauthorized aid on this assignment

In [40]:
import sys

In [41]:
#category 1
cat_1 = '000'
J='000'
BEQ='001'
BNE='010'
BGTZ='011'
SW='100'
LW='101'
BREAK='110'
cat1 = [cat_1,J,BEQ,BNE,BGTZ,SW,LW,BREAK]

In [42]:
#category 2
cat_2 = '001'
ADD='000'
SUB='001'
AND='010'
OR='011'
SRL='100'
SRA='101'
MUL='110'
cat2 = [cat_2,ADD,SUB,AND,OR,SRL,SRA,MUL]

In [43]:
#category 3
cat_3 = '010'
ADDI='000'
ANDI='001'
ORI='010'
cat3 = [cat_3,ADDI,ANDI,ORI]

In [44]:
#register arrays, initialized to zero
registers = [0] * 32 #only write
data = [0] * 16 #only read
lock_data = [0] * 16
data_break = 260
break_index = 0

In [45]:
#initialize all registers to 0
def init():
    global registers
    global data
    global lock_data
    registers = [0] * 32 
    for i in range(len(lock_data)):
        data[i] = lock_data[i]

In [46]:
#convert negative two's complement to int
def convert(val):
        tmp = val.strip()
        tmp = ''.join('1' if x == '0' else '0' for x in tmp)
        tmp = int(tmp,2) + 1
        return tmp*-1

In [47]:
#convert two's complement binary to int
def twos_complement(sample):
        if(sample[0]=='1'): #negative bit
            return convert(sample)

        elif(sample[0]=='0'): #positive bit
            return int(sample,2)

In [48]:
#read in instructions from file, return instruction array
def read_instr(file):
    input_instr = []
    f_dis = open(file,"r")
    for x in f_dis:
        input_instr.append(x)
    return input_instr

In [49]:
#read data from instr
def read_data(input_instr):
    global lock_data
    global data_break
    global break_index
    address = 260
    #data registers, initialized to zero
    broken = False
    
    for index in range(len(input_instr)):
        instr = input_instr[index]
        input_cat = instr[:6]
        if( input_cat == cat_1 + BREAK and not broken):
            break_index = index + 1 # +! because current instruction is break, not a read in value
            broken = True
            data_break = address + 4 #with sample this should result in 316, but change otherwise
        address = address + 4
        
    for index in range(len(input_instr) - break_index-1):
        data[index] = twos_complement(input_instr[index+break_index])
    lock_data = [i for i in data]
    return data

In [50]:
#category 1 function
def cat_1_func(sample,break_exc):

    input_cat = sample[0:3]
    opcode = sample[3:6]

    rs = int(sample[6:11],2)
    rt = int(sample[11:16],2)

    offset = sample[16:32]
    branch_offset = offset
    offset = offset.strip() + '00'

    output = ""
    branch = 0

    if(opcode == J):
        instr_index = sample[8:]
        instr_index = instr_index.strip() + '00'
        instr_index_int = int(instr_index, 2)
        output = "J #" + str(instr_index_int)
        branch = instr_index_int

    if(opcode == BEQ):
        output = "BEQ "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + rt_string + offset_string
        if( registers[rs] == registers[rt] ):
            branch = int(offset,2)

    if(opcode == BNE):
        output = "BNE "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + rt_string + offset_string
        if( registers[rs] != registers[rt] ):
            branch = int(offset,2)


    if(opcode == BGTZ):
        output = "BGTZ "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt) + ", "
        offset_string = "#" + str(int(offset,2))
        output = output + rs_string + offset_string
        if( registers[rs] > 0 ):
            branch = int(offset,2)


    if(opcode == SW): #stores into data
        #dest = rs
        #source = rt
        output = "SW "
        mem = int(sample[16:32],2)
        rs_string = "R" + str(rs)
        rt_string = "R" + str(rt) + ", "
        output = output + rt_string + str(mem) + "(" + rs_string + ")"
        data_index = int(((int(branch_offset,2)-data_break) + registers[rs])/4)
        data[data_index] = registers[rt]

    if(opcode == LW): #loads into registers
        #dest = rt
        #source = rs
        output = "LW "
        mem = int(sample[16:32],2)
        rs_string = "R" + str(rs)
        rt_string = "R" + str(rt) + ", "
        output = output + rt_string + str(mem) + "(" + rs_string + ")"
        #registers <-- from data
        data_index = int(((int(branch_offset,2)-data_break) + registers[rs])/4)
        registers[rt] = data[data_index]


    if(opcode == BREAK):
        output = "BREAK"
        break_exc = True

    #w_dis.write(output + '\n')
    return (output,break_exc,branch)

In [51]:
#category 2 function
def cat_2_func(sample):
    opcode = sample[3:6]
    rd = int(sample[6:11],2)
    rs = int(sample[11:16],2)
    rt = int(sample[16:21],2)
    output = ""


    if(opcode == ADD):
        output = "ADD "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] + registers[rt]

    if(opcode == SUB):
        output = "SUB "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] - registers[rt]

    if(opcode == AND):
        output = "AND "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] and registers[rt]

    if(opcode == OR):
        output = "OR "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] or registers[rt]

    if(opcode == SRL):
        output = "SRL "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "#" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = shift_l(registers[rs], registers[rt])
        #registers[rd] = registers[rs] >> registers[rt]

        #( rd <-- rs >> rt )

    if(opcode == SRA):
        output = "SRA "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "#" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] >> registers[rt]
        #registers[rd] = registers[rs] >> registers[rt]

        #( rd <-- rs >> rt)

    if(opcode == MUL):
        output = "MUL "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        rt_string = "R" + str(rt)
        output = output + rd_string + rs_string + rt_string
        registers[rd] = registers[rs] * registers[rt]
        #()
    return output    

In [52]:
 #category 3 function
def cat_3_func(sample):
    opcode = sample[3:6]
    rd = int(sample[6:11],2)
    rs = int(sample[11:16],2)
    imm = twos_complement(sample[16:])
    output = ""

    if(opcode == ADDI):
        output = "ADDI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] + imm


    if(opcode == ANDI):
        output = "ANDI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] and imm

    if(opcode == ORI):
        output = "ORI "
        rd_string = "R" + str(rd) + ", "
        rs_string = "R" + str(rs) + ", "
        imm_string = "#" + str(imm)
        output = output + rd_string + rs_string + imm_string
        registers[rd] = registers[rs] or imm

    return output   

In [53]:
def data_print():
    output = "Data\n"
    d_316 = str(data_break) + ":"
    d_348 = str(data_break+32) + ":"

    for i in range(8):
        d_316 = d_316 + "\t" + str(data[i])
        d_348 = d_348 + "\t" + str(data[i+8])

    output = output + d_316 + "\n" + d_348 + "\n\n"
    return output


In [54]:
#register print
def register_print():
    output = "Registers\n"
    r_00 = "R00:"
    r_08 = "R08:"
    r_16 = "R16:"
    r_24 = "R24:"

    for i in range(8):
        r_00 = r_00 + "\t" + str(registers[i])
        r_08 = r_08 + "\t" + str(registers[i+8])
        r_16 = r_16 + "\t" + str(registers[i+16])
        r_24 = r_24 + "\t" + str(registers[i+24])
    output = output + r_00 + "\n" + r_08 + "\n" + r_16 + "\n" + r_24 + "\n\n"    
    return output

In [55]:
class apple:
    def __init__(self, nuts):
        self.nuts = nuts
        
    def getnuts(self):
        return self.nuts
    
    def setnuts(self, value):
        self.nuts = value

In [56]:
class Pipeline:
    def __init__(self, instruction):
        self.instruction = [i for i in instruction]
        self.buf_1 = [0] * 8
        self.buf_2 = [0] * 2
        self.buf_3 = [0] * 2
        self.buf_4 = [0] * 2
        self.buf_5 = [0]
        self.buf_6 = [0]
        self.buf_7 = [0]
        self.buf_8 = [0]
        self.buf_9 = [0]
        self.buf_10 = [0]
        
    def get_instr():
        instr = [i for i in instruction]
        return instr

    
    def IF():
        return

In [57]:
#Disassembler function
def disassembler(input_instr):
    address = 260
    break_exc = False
    w_dis = open("disassembly.txt", "w")

    for instruction in input_instr:
        instr = instruction
        w_dis.write(instruction.strip() + '\t' + str(address).strip() + '\t')

        #decode input
        input_cat = instr[:3]

        if(break_exc):
            output = str(twos_complement(instruction))
            w_dis.write(output + '\n')

        if(input_cat == cat_1 and not break_exc): # branch functions possible
            output, break_exc, _ = cat_1_func(instruction,break_exc)
            w_dis.write(output + '\n')

        if(input_cat == cat_2 and not break_exc):
            output = cat_2_func(instruction)
            w_dis.write(output + '\n')

        if(input_cat == cat_3 and not break_exc):
            output = cat_3_func(instruction)
            w_dis.write(output + '\n')

        address = address + 4

    w_dis.close()
    return True

In [58]:
def main(argv):
    #Read in instructions
    instr = read_instr('sample.txt')
    
    #Read in data from instructions
    data = read_data(instr)
    
    #Write to disassembler file
    disassembler(instr)
    
    #Initialize data and registers
    init()
    
    #TODO: Simulation
    #initialize pipeline
    pipeline = Pipeline(instr)
    print(pipeline.get_instr())


In [59]:
if __name__ == "__main__":
    main(sys.argv[1])

TypeError: get_instr() takes 0 positional arguments but 1 was given